In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from joblib import dump

# read csv
df = pd.read_csv("dataset.csv")
df.columns = df.columns.str.lower()

# convert categorical variables to numerical
df["gender"] = df["gender"].str.lower().map({"m": 1, "f": 0})

# drop nan values
df = df.dropna()

# seperate features and target variable
X = df.drop(columns=["lung_cancer"])
y = df["lung_cancer"]

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# svm model
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# performance evaluation
y_pred = svm_model.predict(X_test)

# classification report
dump(svm_model, "svm_lung_model.joblib")

              precision    recall  f1-score   support

           0       0.52      0.51      0.51       382
           1       0.50      0.51      0.51       368

    accuracy                           0.51       750
   macro avg       0.51      0.51      0.51       750
weighted avg       0.51      0.51      0.51       750

✅ Model başarıyla 'svm_lung_model.joblib' olarak kaydedildi.
